In [15]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython']
pipmain(['install'] + packages)

In [16]:
%%capture
from twython import Twython

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import re
import random

In [17]:
import sys
sys.path.append('../')
from auth import (
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

twitter = Twython(
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

In [18]:
PRINT_LIMIT = 50
FOLLOWER_SAMPLE_LIMIT = 20 # number of followers to randomly sample
WORD_FREQ_LIMIT = 10 # return this number of topics that are most freq

In [19]:
user = "AndrewYang"

In [20]:
followers = twitter.get_followers_list(screen_name = user)['users']
print(followers)

[{'id': 1191656946513793024, 'id_str': '1191656946513793024', 'name': 'Jon 🦉✨', 'screen_name': 'ial1992jon', 'location': 'United States', 'description': '#PeteForPresident | He/Him', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 61, 'friends_count': 312, 'listed_count': 0, 'created_at': 'Tue Nov 05 10:02:18 +0000 2019', 'favourites_count': 364, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 408, 'lang': None, 'status': {'created_at': 'Sat Nov 23 18:23:40 +0000 2019', 'id': 1198306135410790401, 'id_str': '1198306135410790401', 'text': 'RT @IndivisibleTeam: As the first millennial to run for President we can’t wait to talk to @PeteButtigieg about how his experience will imp…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'IndivisibleTeam', 'name': 'Indivisible Guide', 'id': 810590628727914496, 'id_str': '810590628727914496', 'indices': [3

In [21]:
limit = 0
followers_list = []
for follower in followers:
    followers_list.append(follower['screen_name'])
print(followers_list)

['ial1992jon', 'savyoung96', 'wayfarerjeff', 'RainerLusmagi', 'buzzynutkins', 'YolandaGehring', 'DamianSasso', 'Reg_castillo_', 'Yo_Its_Pal', 'S1mba4321', 'Awesomecop99', 'zapinski_', 'ethompson_home', 'maverick_theo', '_geeezy', 'ScubabuffRick', '49erpixie', 'Elisestefnik', 'starving_hippo', 'karam_adelina']


In [22]:
def get_tweets_from(follower_name, results = []):
    try: 
        result = twitter.get_user_timeline(screen_name = follower_name)
        results.extend(result)
        return results
    except:
        return results

In [43]:
def get_tweet_info_from_tweets(results):
    tweets = []
    for tweet in results:
        tweet_map = {'text': tweet['text']}
        tweet_map['id'] = tweet['id']
        tweet_map['date'] = tweet['created_at']
        tweet_map['user'] = tweet['user']['screen_name']
        tweet_hashtags = []
        tweet_urls = []
        tweet_user_mentions = []
        
        entities = tweet['entities']
    
        for hashtags in entities['hashtags']:
            tweet_hashtags.append(hashtags['text'])
        for urls in entities['urls']:
            tweet_urls.append(urls['url'])
        try:
            for media in entities['media']:
                tweet_urls.append(media['url'])
        except:
            pass
        for users in entities['user_mentions']:
            tweet_user_mentions.append(users['screen_name'])
            
        tweet_map["hashtags"] = tweet_hashtags
        tweet_map["urls"] = tweet_urls
        tweet_map["user_mentions"] = tweet_user_mentions
        tweets.append(tweet_map)
        
    return tweets

In [24]:
results = []
for f in followers_list:
    results = get_tweets_from(f, results)
print(results)

[{'created_at': 'Sat Nov 23 18:23:40 +0000 2019', 'id': 1198306135410790401, 'id_str': '1198306135410790401', 'text': 'RT @IndivisibleTeam: As the first millennial to run for President we can’t wait to talk to @PeteButtigieg about how his experience will imp…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'IndivisibleTeam', 'name': 'Indivisible Guide', 'id': 810590628727914496, 'id_str': '810590628727914496', 'indices': [3, 19]}, {'screen_name': 'PeteButtigieg', 'name': 'Pete Buttigieg', 'id': 226222147, 'id_str': '226222147', 'indices': [91, 105]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1191656946513793024, 'id_str': '1191656946513793024', 'name': 'Jon 🦉✨', 'screen_name': 'ial1992jon', 'l

In [44]:
tweets_data = get_tweet_info_from_tweets(results)
tweets_data

[{'text': 'RT @IndivisibleTeam: As the first millennial to run for President we can’t wait to talk to @PeteButtigieg about how his experience will imp…',
  'id': 1198306135410790401,
  'date': 'Sat Nov 23 18:23:40 +0000 2019',
  'user': 'ial1992jon',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['IndivisibleTeam', 'PeteButtigieg']},
 {'text': 'RT @BidenExpress: Can’t give your message with out shading other candidates huh? Typical. https://t.co/jL15dCDDzH',
  'id': 1198306069769900032,
  'date': 'Sat Nov 23 18:23:25 +0000 2019',
  'user': 'ial1992jon',
  'hashtags': [],
  'urls': ['https://t.co/jL15dCDDzH'],
  'user_mentions': ['BidenExpress']},
 {'text': 'RT @noah_sissoko: How was the event yesterday in St. Louis you ask?\n\nFull of applause breaks.... https://t.co/8gQgvcuaLH',
  'id': 1198306033921249280,
  'date': 'Sat Nov 23 18:23:16 +0000 2019',
  'user': 'ial1992jon',
  'hashtags': [],
  'urls': ['https://t.co/8gQgvcuaLH'],
  'user_mentions': ['noah_sissoko']},
 {'text': 'R

[NLTK Stopwords](https://pythonspot.com/nltk-stop-words/)

In [26]:
STOPWORDS = stopwords.words('english') + stopwords.words('spanish')
# TODO: decide whether or not to remove tweets that are from retweets altogether
more_stop_words = ['rt']
STOPWORDS += more_stop_words
STOPWORDS

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
def word_extraction(tweet_dictionary, stopwords = STOPWORDS):  
    sentence = tweet_dictionary['text']
    hashtags = ['#' + s for s in tweet_dictionary['hashtags']]
    urls = tweet_dictionary['urls']
    user_mentions = ['@' + s for s in tweet_dictionary['user_mentions']]
    
    ignore = hashtags + urls + user_mentions

    words = sentence.split()  
    
    cleaned_text = []
    
    for word in words:
        cleaned_word = re.sub("[^\w]", "",  word)
        if (not word in ignore) and (not word.lower() in stopwords) and (word.isalpha()) and (len(cleaned_word) > 0):
            cleaned_text.append(cleaned_word.lower())
    return cleaned_text

In [28]:
test_tweet = tweets_data[0]
print(word_extraction(test_tweet))
print(test_tweet)

['first', 'millennial', 'run', 'president', 'wait', 'talk', 'experience']
{'text': 'RT @IndivisibleTeam: As the first millennial to run for President we can’t wait to talk to @PeteButtigieg about how his experience will imp…', 'id': 'RT @IndivisibleTeam: As the first millennial to run for President we can’t wait to talk to @PeteButtigieg about how his experience will imp…', 'date': 'Sat Nov 23 18:23:40 +0000 2019', 'user': 'ial1992jon', 'hashtags': [], 'urls': [], 'user_mentions': ['IndivisibleTeam', 'PeteButtigieg']}


In [29]:
tweets_text = []
for tweet in tweets_data:
    tweets_text.extend(word_extraction(tweet))
print("Extracted words:", tweets_text)

Extracted words: ['first', 'millennial', 'run', 'president', 'wait', 'talk', 'experience', 'give', 'message', 'shading', 'candidates', 'event', 'yesterday', 'louis', 'full', 'applause', 'like', 'people', 'like', 'possible', 'standing', 'wearing', 'wedding', 'rivals', 'call', 'naive', 'people', 'knew', 'young', 'pete', 'audacious', 'run', 'like', 'got', 'afghanistan', 'though', 'oliver', 'davis', 'run', 'mayor', 'south', 'pete', 'endorses', 'davis', 'campaign', 'sets', 'tone', 'encounters', 'buttigieg', 'folks', 'nothing', 'retweet', 'even', 'discredit', 'spreading', 'spread', 'positive', 'messages', 'without', 'repeating', 'polls', 'shows', 'buttigieg', 'percent', 'also', 'garnering', 'percent', 'african', 'american', 'preview', 'general', 'debate', 'would', 'look', 'like', 'leave', 'instead', 'saving', 'pete', 'reaction', 'african', 'american', 'council', 'member', 'south', 'bend', 'endorsing', 'joe', 'biden', 'south', 'bend', 'councilman', 'oliver', 'davis', 'endorses', 'mayor', 'thi

In [39]:
tweets_hashtags = []
for tweet in tweets_data:
    tweets_hashtags.extend(set(tweet['hashtags']))
print("Extracted hashtags:", tweets_hashtags)

Extracted hashtags: ['IStandWithTaylor', 'td', 'vanlife', 'skoolie', 'overlanding', 'rvlife', 'truckcamperlife', 'religiousabuseisecumenical', 'religiouseducationreform', 'resismolech', 'UFC244', 'UFC245', 'buzzards', 'pcbuildingseason', 'O11DynamicXL', 'lianli', 'KickChildhoodCancer', 'TeenChoice', 'ChoiceGamer', 'inspired', 'MAGA', 'dumptrump', 'vusithembekwayo', 'Vusi', 'Domkop', 'RoyalVisitSouthAfrica', 'BULLYSZN', 'PCAs', 'KNICKS', 'JesusIsKing', 'JesusIsKing', 'DubNation', 'saltytweet', 'yankees', 'CFAPeachBowl', 'GoBuffs', 'CFAPeachBowl', 'GoBuffs', 'GoBuffs', 'Leader4ff', 'SportsReport', 'LeaderofthePac', 'HoneymoonBehavior', 'boycottmsnbc', 'fuckMsnbc', 'fuckMsnbc', 'fuckMsnbc', 'fuckMsnbc', 'fuckMsnbc', 'fuckMsnbc']


In [40]:
def get_freq_map(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

def get_top_freq(my_list, top_count):
    freq = get_freq_map(my_list)
    freq = sorted(freq.items(), key=lambda item: item[1], reverse = True)
    
    if len(freq) <= top_count:
        return freq
    else:
        return freq[:10]

In [41]:
tweets_text_freq = get_top_freq(tweets_text, WORD_FREQ_LIMIT)
print("top {0} most freq words:".format(WORD_FREQ_LIMIT))
tweets_text_freq

top 10 most freq words:


[('like', 20),
 ('go', 15),
 ('get', 12),
 ('never', 8),
 ('president', 7),
 ('people', 7),
 ('even', 7),
 ('would', 7),
 ('back', 7),
 ('going', 7)]

In [42]:
tweets_hashtags_freq = get_top_freq(tweets_hashtags, WORD_FREQ_LIMIT)
print("top {0} most freq hashtags:".format(WORD_FREQ_LIMIT))
tweets_hashtags_freq

top 10 most freq hashtags:


[('fuckMsnbc', 6),
 ('GoBuffs', 3),
 ('JesusIsKing', 2),
 ('CFAPeachBowl', 2),
 ('IStandWithTaylor', 1),
 ('td', 1),
 ('vanlife', 1),
 ('skoolie', 1),
 ('overlanding', 1),
 ('rvlife', 1)]